In [ ]:
!pip install -U -q google-generativeai

In [ ]:
#Importações e configurações iniciais
import numpy as np
import pandas as pd
import google.generativeai as genai

from google.colab import userdata
api_key = userdata.get('secretName')

genai.configure(api_key=api_key)

In [ ]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [ ]:
print("O que fazer após uma parada cardiorrespiratória???")

O que fazer após uma parada cardiorrespiratória???


In [ ]:
#Listagem de documentos que serão buscados
DOCUMENT1 = {
    "Título": "Reanimação cardiopulmonar (RCP)",
    "Conteúdo": "#Atenção!!! Em primeiro lugar, chamar uma ambulância, ligando para o 192. #30 compressões torácicas à frequência de 100 a 120 massagens por minuto (2 por segundo). #em seguinte, abertura da via respiratória com elevação do queixo e inclinação da testa para trás. #duas respirações boca-a-boca"}

DOCUMENT2 = {
    "Título": "Como fazer uma Massagem cardíaca:",
    "Conteúdo": "#Colocar as duas mãos no centro do peito com os dedos entrelaçados, no ponto médio entre os mamilos; #Fazer compressões mantendo os braços esticados e empurrando o peito para baixo até que as costelas desçam cerca de 5 cm; #Manter as compressões até a chegada da ajuda médica num ritmo de 2 compressões por segundo. #O ciclo de compressões e respirações continua até a chegada de profissionais de atendimento e que seja providenciado um desfibrilador."}

documents = [DOCUMENT1, DOCUMENT2]

In [ ]:
df = pd.DataFrame(documents)
df.columns = ["Titulo", "Conteudo"]
df

,Titulo,Conteudo
0,Reanimação cardiopulmonar (RCP),"#Atenção!!! Em primeiro lugar, chamar uma ambu..."
1,Como fazer uma Massagem cardíaca:,#Colocar as duas mãos no centro do peito com o...


In [ ]:
model = "models/embedding-001"

In [ ]:
def embed_fn(title, text):
  return genai.embed_content(model=model,
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [ ]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"], row["Conteudo"]), axis=1)
df

,Titulo,Conteudo,Embeddings
0,Reanimação cardiopulmonar (RCP),"#Atenção!!! Em primeiro lugar, chamar uma ambu...","[0.007867859, -0.045537047, -0.021149471, 0.05..."
1,Como fazer uma Massagem cardíaca:,#Colocar as duas mãos no centro do peito com o...,"[0.032632217, -0.052456357, -0.016461607, 0.03..."


In [ ]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Conteudo"]

In [ ]:
consulta = "O que fazer após uma parada cardiorrespiratória? Como Executar a Reanimação cardiopulmonar (RCP) e como fazer a massagem cardíaca?"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

#Atenção!!! Em primeiro lugar, chamar uma ambulância, ligando para o 192. #30 compressões torácicas à frequência de 100 a 120 massagens por minuto (2 por segundo). #em seguinte, abertura da via respiratória com elevação do queixo e inclinação da testa para trás. #duas respirações boca-a-boca


In [ ]:
generation_config = {
  "temperature": 0,
  "candidate_count": 1
}

In [ ]:
prompt = f"Crie um passo a passo objetivo, em forma de checklist, como no mínimo 7 passos, e sem adicionar informações que não façam parte do texto, de como executar a eeanimação cardiopulmonar (RCP) e como fazer a massagem cardíaca: {trecho}"

model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)

**Checklist de RCP e Massagem Cardíaca**

1. **Chame uma ambulância:** Ligue para o 192 imediatamente.
2. **Verifique a segurança:** Certifique-se de que o local é seguro para você e a vítima.
3. **Verifique a resposta:** Toque na vítima e grite seu nome. Se não houver resposta, prossiga.
4. **Abra a via respiratória:** Incline a cabeça da vítima para trás e levante seu queixo.
5. **Verifique a respiração:** Olhe, ouça e sinta a respiração da vítima por até 10 segundos.
6. **Inicie as compressões torácicas:**
    * Coloque as mãos sobre o centro do peito da vítima.
    * Comprima o peito a uma profundidade de 5 a 6 cm.
    * Faça 30 compressões à frequência de 100 a 120 por minuto (2 por segundo).
7. **Dê respirações boca-a-boca:**
    * Após 30 compressões, incline a cabeça da vítima para trás e levante seu queixo.
    * Feche o nariz da vítima e cubra sua boca com a sua.
    * Dê duas respirações de resgate, cada uma com duração de cerca de 1 segundo.
